In [3]:
import requests
import json
import keyring
import pandas as pd
import time
import numpy as np
import datetime
from datetime import timedelta
import schedule
# 한국 시간
import pytz


#### 포트폴리오 매수

In [5]:
# API Key
app_key = keyring.get_password('mock_app_key', '@2229673')
app_secret = keyring.get_password('mock_app_secret', '@2229673')

# base url
url_base = "https://openapivts.koreainvestment.com:29443" # 모의투자

# information
headers = {"content-type": "application/json"}
path = "oauth2/tokenP"
body = {
    "grant_type": "client_credentials",
    "appkey": app_key,
    "appsecret": app_secret
}

url = f"{url_base}/{path}"
print(url)

https://openapivts.koreainvestment.com:29443/oauth2/tokenP


In [6]:
res = requests.post(url, headers=headers, data=json.dumps(body))
access_token = res.json()['access_token']

In [8]:
# 현재가 구하기
def get_price(ticker):
    path = "uapi/domestic-stock/v1/quotations/inquire-price"
    url = f"{url_base}/{path}"

    headers = {
        "Content-Type": "application/json",
        "authorization": f"Bearer {access_token}",
        "appKey": app_key,
        "appSecret": app_secret,
        "tr_id": "FHKST01010100"
    }

    params = {"fid_cond_mrkt_div_code": "J", "fid_input_iscd": ticker}

    res = requests.get(url, headers=headers, params=params)
    price = res.json()['output']['stck_prpr']
    price = int(price)
    time.sleep(0.1)

    return price

In [9]:
def hashkey(datas):
    path = "uapi/hashkey"
    url = f"{url_base}/{path}"
    headers = {
        'content-Type': 'application/json',
        'appKey': app_key,
        'appSecret': app_secret,
    }
    res = requests.post(url, headers=headers, data=json.dumps(datas))
    hashkey = res.json()["HASH"]

    return hashkey

In [10]:
# 주문
def trading(ticker, tr_id):

    path = "/uapi/domestic-stock/v1/trading/order-cash"
    url = f"{url_base}/{path}"

    data = {
        "CANO": "50102559", # 계좌번호 앞 8지리
        "ACNT_PRDT_CD": "01",
        "PDNO": ticker,     # 종목코드
        "ORD_DVSN": "03",   # 주문 방법
        "ORD_QTY": "1",     # 주문 수량
        "ORD_UNPR": "0",    # 주문 단가 (시장가의 경우 0)
    }

    headers = {
        "Content-Type": "application/json",
        "authorization": f"Bearer {access_token}",
        "appKey": app_key,
        "appSecret": app_secret,
        "tr_id": tr_id,
        "custtype": "P",
        "hashkey": hashkey(data)
    }

    res = requests.post(url, headers=headers, data=json.dumps(data)) 

In [11]:
# 계좌 잔고 조회
def check_account():

    output1 = []
    output2 = []
    CTX_AREA_NK100 = ''

    while True:

        path = "/uapi/domestic-stock/v1/trading/inquire-balance"
        url = f"{url_base}/{path}"

        headers = {
            "Content-Type": "application/json",
            "authorization": f"Bearer {access_token}",
            "appKey": app_key,
            "appSecret": app_secret,
            "tr_id": "VTTC8434R"
        }

        params = {
            "CANO": "50102559", # 계좌번호 앞 8지리
            "ACNT_PRDT_CD": "01",
            "AFHR_FLPR_YN": "N",
            "UNPR_DVSN": "01",
            "FUND_STTL_ICLD_YN": "N",
            "FNCG_AMT_AUTO_RDPT_YN": "N",
            "OFL_YN": "",
            "INQR_DVSN": "01",
            "PRCS_DVSN": "00",
            "CTX_AREA_FK100": '',
            "CTX_AREA_NK100": CTX_AREA_NK100
        }

        res = requests.get(url, headers=headers, params=params)
        output1.append(pd.DataFrame.from_records(res.json()['output1']))

        CTX_AREA_NK100 = res.json()['ctx_area_nk100'].strip()

        if CTX_AREA_NK100 == '':
            output2.append(res.json()['output2'][0])
            break

    if not output1[0].empty:
        res1 = pd.concat(output1)[['pdno',
                                   'hldg_qty']].rename(columns={
                                       'pdno': '종목코드',
                                       'hldg_qty': '보유수량'
                                   }).reset_index(drop=True)
    else:
        res1 = pd.DataFrame(columns=['종목코드', '보유수량'])

    res2 = output2[0]

    return [res1, res2]

이제 각 종목 당 몇주를 사야하는지 수량을 계산하자.

In [12]:
# 모델 포트폴리오
mp = pd.DataFrame({
    '종목코드': [
        '005930',  # 삼성전자
        '373220',  # LG에너지솔루션
        '000660',  # SK하이닉스
        '207940',  # 삼성바이오로직스
        '051910',  # LG화학
        '035420',  # NAVER
        '005380',  # 현대차
        '006400',  # 삼성SDI,
        '035720',  # 카카오
        '105560',  #KB금융
    ]
})

In [13]:
# 보유 종목과 aum 불러오기
ap, account = check_account()

In [16]:
# 주당 투자 금액
invest_per_stock = int(account['tot_evlu_amt']) * 0.98 / len(mp)

In [22]:
# 매매 구성
target = mp.merge(ap, on='종목코드', how='outer')
target['보유수량'] = target['보유수량'].fillna(0).apply(pd.to_numeric)

C:\Users\ewp\AppData\Local\Temp\ipykernel_10428\1193563776.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  target['보유수량'] = target['보유수량'].fillna(0).apply(pd.to_numeric)


In [33]:
# 현재가 확인
target['현재가'] = target.apply(lambda x: get_price(x.종목코드), axis=1)

In [42]:
# 목표수량 및 투자수량 입력
target['목표수량'] = np.where(target['종목코드'].isin(mp['종목코드'].tolist()),
                          round(invest_per_stock / target['현재가']), 0)

In [46]:
target['투자수량'] = target['목표수량'] - target['보유수량']

In [47]:
target

,종목코드,보유수량,현재가,목표수량,투자수량
0,000660,0,171900,6.0,6.0
1,005380,0,253000,4.0,4.0
2,005930,0,73300,13.0,13.0
3,006400,0,426000,2.0,2.0
4,035420,0,188000,5.0,5.0
5,035720,0,53200,18.0,18.0
6,051910,0,443000,2.0,2.0
7,105560,0,70900,14.0,14.0
8,207940,0,838000,1.0,1.0
9,373220,0,402500,2.0,2.0


In [50]:
# Define the Korea Standard Time timezone
kst_tz = pytz.timezone('Asia/Seoul')

# 시간 분할
startDt1 = datetime.datetime.now().astimezone(kst_tz) + timedelta(minutes=1)
startDt2 = datetime.datetime.now().astimezone(kst_tz).replace(hour=9,minute=10,second=0,microsecond=0)
startDt = max(startDt1, startDt2)
endDt = datetime.datetime.now().astimezone(kst_tz).replace(hour=15,minute=0,second=0,microsecond=0)

In [116]:
# 스케줄 초기화
schedule.clear()

In [52]:
# 스케줄 등록
for t in range(target.shape[0]) :
    
    n = target.loc[t, '투자수량']                    # Define quantity
    position = 'VTTC0802U' if n > 0 else 'VTTC0801U' # Sell: VTTC0802U or Buy: VTTC0802U
    ticker = target.loc[t, '종목코드']                # Define ticker 

    time_list = pd.date_range(startDt, endDt, periods = abs(n))    
    time_list = time_list.round(freq = 's').tolist()    
    time_list_sec = [s.strftime('%H:%M:%S') for s in time_list]                 

    for i in time_list_sec:
        schedule.every().day.at(i).do(trading, ticker, position) 

C:\Users\ewp\AppData\Local\Temp\ipykernel_10428\2448283365.py:8: FutureWarning: Non-integer 'periods' in pd.date_range, pd.timedelta_range, pd.period_range, and pd.interval_range are deprecated and will raise in a future version.
  time_list = pd.date_range(startDt, endDt, periods = abs(n))
C:\Users\ewp\AppData\Local\Temp\ipykernel_10428\2448283365.py:8: FutureWarning: Non-integer 'periods' in pd.date_range, pd.timedelta_range, pd.period_range, and pd.interval_range are deprecated and will raise in a future version.
  time_list = pd.date_range(startDt, endDt, periods = abs(n))
C:\Users\ewp\AppData\Local\Temp\ipykernel_10428\2448283365.py:8: FutureWarning: Non-integer 'periods' in pd.date_range, pd.timedelta_range, pd.period_range, and pd.interval_range are deprecated and will raise in a future version.
  time_list = pd.date_range(startDt, endDt, periods = abs(n))
C:\Users\ewp\AppData\Local\Temp\ipykernel_10428\2448283365.py:8: FutureWarning: Non-integer 'periods' in pd.date_range, pd.t

In [53]:
# 스케줄 확인
schedule.get_jobs()

[Every 1 day at 14:40:05 do trading('000660', 'VTTC0802U') (last run: [never], next run: 2024-03-10 14:40:05),
 Every 1 day at 14:44:04 do trading('000660', 'VTTC0802U') (last run: [never], next run: 2024-03-10 14:44:04),
 Every 1 day at 14:48:03 do trading('000660', 'VTTC0802U') (last run: [never], next run: 2024-03-10 14:48:03),
 Every 1 day at 14:52:02 do trading('000660', 'VTTC0802U') (last run: [never], next run: 2024-03-10 14:52:02),
 Every 1 day at 14:56:01 do trading('000660', 'VTTC0802U') (last run: [never], next run: 2024-03-10 14:56:01),
 Every 1 day at 15:00:00 do trading('000660', 'VTTC0802U') (last run: [never], next run: 2024-03-10 15:00:00),
 Every 1 day at 14:40:05 do trading('005380', 'VTTC0802U') (last run: [never], next run: 2024-03-10 14:40:05),
 Every 1 day at 14:46:43 do trading('005380', 'VTTC0802U') (last run: [never], next run: 2024-03-10 14:46:43),
 Every 1 day at 14:53:22 do trading('005380', 'VTTC0802U') (last run: [never], next run: 2024-03-10 14:53:22),
 

In [114]:
# 스케줄 실행
while True:
    schedule.run_pending()    
    if datetime.datetime.now().astimezone(kst_tz) > endDt :
        print('거래가 완료되었습니다.')        
        schedule.clear()
        break

KeyboardInterrupt: 

#### 포트폴리오 리밸런싱

---

In [ ]:
import pytz
# Define the Korea Standard Time timezone
kst_tz = pytz.timezone('Asia/Seoul')

In [80]:
# 진입: 


# 일단 오늘이 트레이딩 할 날인지: 
# 만약 맞으면: 
# 리벨런싱 

# 아니면: 
# 아무것도 안함 

In [83]:
today = datetime.datetime.now().astimezone(kst_tz)

In [84]:
today

datetime.datetime(2024, 3, 11, 7, 6, 24, 980847, tzinfo=<DstTzInfo 'Asia/Seoul' KST+9:00:00 STD>)

#### 날짜

In [70]:
from pandas_market_calendars import get_calendar

In [79]:
exchange_name = 'NYSE'
exchange_calendar = get_calendar(exchange_name)
today = datetime.datetime.now()

In [73]:
exchange_calendar

In [54]:
def is_business_day(date):
    return pd.to_datetime(date).weekday() < 5

In [55]:
today = datetime.datetime.now()

In [57]:
is_business_day(today.astimezone(kst_tz))

True

In [59]:
def last_business_day_of_month(year, month):
    end_of_month = datetime.date(year, month, 1) + pd.offsets.MonthEnd(0)
    while not is_business_day(end_of_month):
        end_of_month -= datetime.timedelta(days=1)
    return end_of_month

In [62]:
today.astimezone(kst_tz).date().month

3

In [65]:
datetime.date(today.astimezone(kst_tz).date().year, today.astimezone(kst_tz).date().month+2, 1) + pd.offsets.MonthEnd(0)

Timestamp('2024-05-31 00:00:00')

---

#### 주식 현재가 시세 조회하기

In [23]:
path = "uapi/domestic-stock/v1/quotations/inquire-price"
url = f"{url_base}/{path}"

headers = {
    "Content-Type": "application/json",
    "authorization": f"Bearer {access_token}",
    "appKey": app_key,
    "appSecret": app_secret,
    "tr_id": "FHKST01010100"
}

# fid_input_iscd: 티커
params = {"fid_cond_mrkt_div_code": "J", "fid_input_iscd": "005930"}   

res = requests.get(url, headers=headers, params=params)
res.json()['output']['stck_prpr']

'73400'

#### 주식 잔고조회

In [25]:
path = "/uapi/domestic-stock/v1/trading/inquire-balance"
url = f"{url_base}/{path}"

headers = {
    "Content-Type": "application/json",
    "authorization": f"Bearer {access_token}",
    "appKey": app_key,
    "appSecret": app_secret,
    "tr_id": "VTTC8434R"
}

params = {
    "CANO": "50102559",  # 계좌번호 앞 8지리
    "ACNT_PRDT_CD": "01",  # 계좌번호 뒤 2자리
    "AFHR_FLPR_YN": "N",  # 시간외단일가여부
    "OFL_YN": "",  # 공란
    "INQR_DVSN": "01",  # 조회구분
    "UNPR_DVSN": "01",  # 단가구분
    "FUND_STTL_ICLD_YN": "N",  # 펀드결제분포함여부
    "FNCG_AMT_AUTO_RDPT_YN": "N",  # 융자금액자동상환여부        
    "PRCS_DVSN": "00",  # 처리구분(00: 전일매매포함)
    "CTX_AREA_FK100": "",  # 연속조회검색조건
    "CTX_AREA_NK100": ""  # 연속조회키
}

res = requests.get(url, headers=headers, params=params)

In [27]:
res.json()['output1']

[]

In [28]:
res.json()['output2']

[{'dnca_tot_amt': '10000000',
  'nxdy_excc_amt': '10000000',
  'prvs_rcdl_excc_amt': '10000000',
  'cma_evlu_amt': '0',
  'bfdy_buy_amt': '0',
  'thdt_buy_amt': '0',
  'nxdy_auto_rdpt_amt': '0',
  'bfdy_sll_amt': '0',
  'thdt_sll_amt': '0',
  'd2_auto_rdpt_amt': '0',
  'bfdy_tlex_amt': '0',
  'thdt_tlex_amt': '0',
  'tot_loan_amt': '0',
  'scts_evlu_amt': '0',
  'tot_evlu_amt': '10000000',
  'nass_amt': '10000000',
  'fncg_gld_auto_rdpt_yn': '',
  'pchs_amt_smtl_amt': '0',
  'evlu_amt_smtl_amt': '0',
  'evlu_pfls_smtl_amt': '0',
  'tot_stln_slng_chgs': '0',
  'bfdy_tot_asst_evlu_amt': '10000000',
  'asst_icdc_amt': '0',
  'asst_icdc_erng_rt': '0.00000000'}]

#### 매수 주문

In [24]:
path = "/uapi/domestic-stock/v1/trading/order-cash"
url = f"{url_base}/{path}"

data = {
    "CANO": " 50102559",  # 계좌번호 앞 8지리
    "ACNT_PRDT_CD": "01",  # 계좌번호 뒤 2자리
    "PDNO": "005930",  # 종목코드
    "ORD_DVSN": "01",  # 주문 방법
    "ORD_QTY": "10",  # 주문 수량
    "ORD_UNPR": "0",  # 주문 단가 (시장가의 경우 0)
}

headers = {
    "Content-Type": "application/json",
    "authorization": f"Bearer {access_token}",
    "appKey": app_key,
    "appSecret": app_secret,
    "tr_id": "VTTC0802U",
    "custtype": "P",
    "hashkey": hashkey(data)
}

res = requests.post(url, headers=headers, data=json.dumps(data))
res.json()

{'rt_cd': '1',
 'msg_cd': 'IGW00002',
 'msg1': '인증 시점의 계좌번호와 요청 계좌번호가 일치하지 않습니다.'}